In [1]:
try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [2]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [3]:
# import local files
import main
import indices
import utils
import mask

### General Workflow

1. Import the area of interest (soum shapefile is first uploaded to GEE asset)
2. Import mask areas (Soum Seasonal Range Areas)
3. Filter data
    - Date
    - Cloud Coverage
    - Best pixel
4. Apply functions
    - Landsat sensors harmonization
    - Cloudless pixels
    - Vegetation indices
    - Custom masks
5. Download datasets per month and year per soum 


In [4]:
# Import area of interest
# bags = ee.FeatureCollection('users/ta346/mng_grassland/mng_nso_bag')
soums = ee.FeatureCollection('users/ta346/mng_boundary_cleaned/mn_soum_cln')

In [5]:
#import seasonal
# winter - 1, summer - 2, pasture-not-used - 3
pasture = ee.Image("users/ta346/pasture_delineation/pas_raster_new")
# import landcover (20m) tiff image from asset in Google Earth Engine
# [10, 20, 60, 80, 100]
# search what files are in the GEE asset folder geemap.ee_search()
lc2020 = ee.Image('users/ta346/mng_landcover_30m/2020LC30').clip(soums).select('b1')
lc2010 = ee.Image('users/ta346/mng_landcover_30m/2010LC30').clip(soums).select('b1')
lc2000 = ee.Image('users/ta346/mng_landcover_30m/2000LC30').clip(soums).select('b1')

### 1. Landsat Collection 

In [6]:
# cloud free landsat collection
landsat_collection = main.get_landsat_collection(dateIni='2022-01-01', # initial date
                                                        dateEnd='2023-12-31', # end date
                                                        box=soums, # area of interest
                                                        perc_cover=50, # only images where more than 50% of pixels are cloud free
                                                        sensor=["LC08", "LE07", "LT05"], # search for all available sensors
                                                        harmonization=True) # ETM and ETM+ to OLI

In [7]:
# Compute vegetation indices on cloud free landsat collection
landsat_collection = (landsat_collection.map(indices.ndvi(nir= "SR_B4", red = "SR_B3", bandname = "ndvi"))
                                   .map(indices.evi(nir = "SR_B4", red = "SR_B3", blue = "SR_B1", G = 2.5, C1 = 6, C2 = 7.5, L=1, bandname='evi'))
                                   .map(indices.savi(nir = "SR_B4", red = "SR_B3", L = 0.5, G = 1.5, bandname="savi"))
                                   .map(indices.msavi(nir = "SR_B4", red = "SR_B3", G = 2, H = 8, L = 1, bandname="msavi"))
                                   .map(indices.nirv(nir = "SR_B4", red = "SR_B3", bandname="nirv"))
                                   .map(indices.ndwi(nir = "SR_B4", swir = "SR_B5", bandname="ndwi")))
# vegetation index interests 
index_need_landsat = ['ndvi', 'evi', 'savi', 'msavi', 'nirv', 'ndwi']

In [8]:
# save each resulting CSV's to Google Drive Folder 
# this exports landsat collection per year on three different sceneries:
    # a. winter grazing ranges (wgr)
    # b. summer grazing ranges (sgr)
    # c. both grazing ranges (gr)
# create a list of names for mask
download = ['sgr']

for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = "MNG_" + str(i).upper() + "_" + "VEGI_LANDSAT_30m"
    folder_name = str(i).upper() + "_" + "SOUM_LANDSAT_VEG"

    # download each image per year as csv file over area of interest at soum level
    comp = main.download_img_col_to_csv_monthly(landsat_collection, 
                                                    startYear = 2022,
                                                    endYear = 2023,
                                                    startMonth = 1,
                                                    endMonth = 12,
                                                    bandnames = index_need_landsat, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 30,
                                                    tileScale = 1,
                                                    other_mask = other_mask,
                                                    other_mask_parameter = other_mask_parameter,  
                                                    crs = "EPSG:4326", 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

### 2. MODIS43A

In [13]:
# modis46A cloudless image collection
modis46a = main.get_modis46a_500_collection(dateIni='2022-01-01', 
                                                dateEnd='2023-12-31', 
                                                box = soums, 
                                                quality_mask=True)

In [14]:
# Compute vegetation indices on cloud free landsat collection
modis46a = (modis46a.map(indices.ndvi(nir = "Nadir_Reflectance_Band2", red = 'Nadir_Reflectance_Band1', bandname='ndvi'))
                .map(indices.evi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', blue = 'Nadir_Reflectance_Band3', bandname='evi'))
                .map(indices.savi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='savi'))
                .map(indices.msavi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='msavi'))
                .map(indices.nirv(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='nirv')))
# vegetation index interests 
index_need_modis = ['ndvi', 'evi', 'savi', 'msavi', 'nirv']

In [8]:
# create a list of names for mask
download = ['sgr']

# save each image collection per year
for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = "MNG" + "_" + str(i).upper() + "_" + "VEGI_MODIS_500m"
    folder_name = str(i).upper() + "_" + "SOUM_MODIS_VEG"

    # download each image per year as csv file over area of interest at soum level
    comp = main.download_img_col_to_csv_monthly(modis46a, 
                                                    startYear = 2022,
                                                    endYear = 2023,
                                                    startMonth = 1,
                                                    endMonth = 12,
                                                    bandnames = index_need_modis, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 500,
                                                    tileScale = 1,
                                                    other_mask = other_mask,
                                                    other_mask_parameter = other_mask_parameter,  
                                                    crs = "EPSG:4326", 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

### 3. Weather

In [ ]:
download = ['sgr', 'wgr']

In [ ]:
# band names
select_band = ["temperature_2m", 
               "snow_cover", 
               "snow_density", 
               "snow_depth", 
               "snow_depth_water_equivalent", 
               "snowfall", 
               "snowmelt", 
               "u_component_of_wind_10m", 
               "v_component_of_wind_10m", 
               "total_precipitation"]

In [ ]:
# get era5 collection
era5 = main.get_era5_collection('2022-01-01', '2023-12-31', soums, select_band)

In [ ]:
# save each image collection as CSV's per month to Google Drive Folder 
for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = str(i) + "_" + "weather"
    folder_name = str(i).upper() + "_" + "SOUM_ERA_WEATHER"

    # download each image per year as csv file over area of interest at soum level
    comp = main.download_img_col_to_csv_monthly(era5, 
                                                    startYear = 2022,
                                                    endYear = 2023,
                                                    startMonth = 1,
                                                    endMonth = 12,
                                                    bandnames = select_band, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 9000,
                                                    tileScale = 1,
                                                    other_mask = other_mask,
                                                    other_mask_parameter = other_mask_parameter,  
                                                    crs = "EPSG:4326", 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

### Buffer Area (56m and 100m) vegitation indices

In [32]:
buffer_100m = ee.FeatureCollection("users/ta346/bm_buffer/bm_buffer_100m")
buffer_50m = ee.FeatureCollection("users/ta346/bm_buffer/bm_buffer_56m")

df = geemap.ee_to_geopandas(buffer_100m, selectors = ['GID', 'asid'])
df1 = geemap.ee_to_geopandas(buffer_50m, selectors = ['GID', 'asid'])

df["gid_asid"] = df['GID']*10000 + df['asid']
df1["gid_asid"] = df1['GID']*10000 + df1['asid']

df.crs = "EPSG:4326"
df1.crs = "EPSG:4326"

buffer_100m = geemap.geopandas_to_ee(df)
buffer_56m = geemap.geopandas_to_ee(df1)

In [33]:
download = ['sgr']

In [35]:
def download_images(collection, box, folder_name, collection_name, buffer_size, scale, index_need):
    comp = main.download_img_col_to_csv_monthly(
        collection,
        startYear=2023,
        endYear=2023,
        startMonth=9,
        endMonth=9,
        bandnames=index_need,
        box=box,
        reducerAll=True,
        feat_name='gid_asid',
        scale=scale,
        tileScale=1,
        crs="EPSG:4326",
        file_name=f"MNG_VEGI_{collection_name}_{scale}m_plot{buffer_size}m",
        folder_name=folder_name
    )
    return comp

# # Download Landsat images for buffer_56m and buffer_100m
download_images(landsat_collection, buffer_56m, folder_name = "PLOT56", collection_name = "LANDSAT", buffer_size = 56, scale = 30, index_need = index_need_landsat)
download_images(landsat_collection, buffer_100m, folder_name = "PLOT100", collection_name = "LANDSAT", buffer_size = 100, scale = 30, index_need = index_need_landsat)

# Download Landsat images for buffer_56m and buffer_100m
# download_images(modis46a, buffer_56m, folder_name = "PLOT56", collection_name = "MODIS", buffer_size = 56, scale = 500, index_need = index_need_modis)
# download_images(modis46a, buffer_100m, folder_name = "PLOT100", collection_name = "MODIS", buffer_size = 100, scale = 500, index_need = index_need_modis)

In [ ]:
import pandas as pd

old = pd.read_csv('C://Users//Owner//Downloads//Telegram Desktop//MNG_VEGI_LANDSAT_30m_plot100m_2022_12.csv')
new = pd.read_csv('C://Users//Owner//Downloads//MNG_VEGI_LANDSAT_30m_plot100m_2022_12_1.csv')

In [ ]:
(old.shape), (new.shape)